Image Generator
===============

An AI assistant using DALL-E-2 or DALL-E-3 for the image generation.

# Dependencies

In [ ]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import base64
from io import BytesIO
from PIL import Image
from IPython.display import display

In [ ]:
# Initialization

output_path = "image.webp"

load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
MODEL = "dall-e-3"
# MODEL = "dall-e-2"

openai = OpenAI()

# Image Generation

In [ ]:
def artist(image_prompt, image_size):
    image_response = openai.images.generate(
            model=MODEL,
            prompt=image_prompt,
            size=image_size,
            n=1,
            response_format="b64_json",
        )
    image_base64 = image_response.data[0].b64_json
    image_data = base64.b64decode(image_base64)
    save_base64_to_webp(image_base64, output_path)
    return Image.open(BytesIO(image_data))

def save_base64_to_webp(base64_string, output_path):
    """
    Decodes a base64 string and saves it as a WebP image.

    Args:
        base64_string: The base64 encoded image data.
        output_path: The path to save the WebP image.
    """
    try:
        image_data = base64.b64decode(base64_string)
        image = Image.open(BytesIO(image_data))
        image.save(output_path, "webp")
        print(f"Image saved as {output_path}")
    except Exception as e:
        print(f"Error saving image: {e}")

In [ ]:
# Gradio code for non-preset Chat interface
sample_image_prompt="An image representing a vacation in London, showing tourist spots and everything unique about London, in a vibrant pop-art style"
    
with gr.Blocks() as ui:
    gr.Markdown("## Create Image")
    with gr.Row():
        image_prompt = gr.Textbox(label="Image prompt:", value=sample_image_prompt, lines=21)
        image_output = gr.Image(height=500)
    with gr.Row():
        if MODEL == "dall-e-2":
            size = gr.Dropdown(["256x256", "512x512", "1024x1024"], label="Select size", value="1024x1024")
        else:
            size = gr.Dropdown(["1024x1024", "1024x1792", "1792x1024"], label="Select size", value="1024x1024")
        create = gr.Button("Generate image", value=output_path, visible=True)

    create.click(artist, inputs=[image_prompt, size], outputs=[image_output])

ui.launch(inbrowser=True)